### Spotify User Artists

This notebook helps you find the top 50 artists you've listened to. To run this notebook, you need the following
libraries. They will be auto-installed by the notebook in your notebook's python kernel.
* python3
* spotipy (https://github.com/plamere/spotipy)
* tabulate (https://github.com/astanin/python-tabulate)
* cachetools (https://github.com/tkem/cachetools/)
* python-dotenv (https://github.com/theskumar/python-dotenv)

You will also need to create your own Spotify application by visiting: https://developer.spotify.com/dashboard/applications.
You need to create a file named *.env* in this folder and add the following lines to it:
```
SPOTIFY_CLIENT_ID=
SPOTIFY_CLIENT_SECRET=
```
where the values are your Spotify application's CLIENT_ID and CLIENT_SECRET.

This notebook will prompt you to login to Spotify in your native browser and grant the following permissions:
* **user-top-read:** Fetch the top artists from Spotify for your user account.
* **user-read-recently-played:** Get 50 recently played songs by your account. We use this to generate your recent top genres.

### Configs
Feel free to update the configs in the following cell.

In [8]:
# Valid values: long_term - calculated from several years of data and including all new data as it becomes available,
#               medium_term - approximately last 6 months,
#               short_term - approximately last 4 weeks
TIME_RANGE = 'long_term'

### Install the required libraries

In [9]:
import sys

!{sys.executable} -m pip install spotipy
!{sys.executable} -m pip install tabulate
!{sys.executable} -m pip install cachetools
!{sys.executable} -m pip install python-dotenv

Load the client id and secret keys from environment.

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()
CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

In [11]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from tabulate import tabulate
from cachetools import cached, TTLCache

Define the auth function. The result is cached for 1 hour to avoid switching to browser everytime you execute the
notebook.

Scopes used in this notebook are documented at: https://developer.spotify.com/documentation/general/guides/scopes

In [12]:
@cached(cache=TTLCache(maxsize=16, ttl=3600))
def auth_for_scope(scope):
    sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               scope=scope,
                                               cache_path='.cache_' + scope,
                                               redirect_uri='http://localhost:8000'))

    return sp

### Top Artists
The next cell lists the top artists for you in the configured time window.

In [13]:
results = auth_for_scope('user-top-read').current_user_top_artists(limit=50, time_range=TIME_RANGE)

output = []
for idx, item in enumerate(results['items']):
    name = item['name']
    popularity = item['popularity']
    genres = item['genres'][:5]
    output.append([name, popularity, genres])

print(tabulate(output, headers=['Name', 'Popularity', 'Genres']))

Name                                  Popularity  Genres
----------------------------------  ------------  -----------------------------------------------------------------------------------------
Pink Floyd                                    84  ['album rock', 'art rock', 'classic rock', 'progressive rock', 'psychedelic rock']
Coldplay                                      89  ['permanent wave', 'pop']
Cat Music                                     60  ['pet calming']
Dream Theater                                 66  ['metal', 'neo classical metal', 'power metal', 'progressive metal', 'progressive rock']
Queen                                         90  ['classic rock', 'glam rock', 'rock']
Keira Knightley                               54  ['hollywood']
The Beatles                                   89  ['beatlesque', 'british invasion', 'classic rock', 'merseybeat', 'psychedelic rock']
The Rolling Stones                            84  ['album rock', 'british invasion', 'classic rock', '

### Top Genres
The next cell lists the top genres you've listened to recently - last 50 songs.

In [14]:
auth = auth_for_scope('user-read-recently-played')
recently_played_tracks = auth.current_user_recently_played()

output = {}
for item in recently_played_tracks['items']:
    for artist in item['track']['artists']:
        for genre in auth.artist(artist['id'])['genres']:
            output[genre] = output.get(genre, 0) + 1

output = sorted([(k, v) for k, v in output.items()], key=lambda x: x[1], reverse=True)

print(tabulate(output, headers=['Gneres', 'Plays']))


Gneres                 Plays
-------------------  -------
rock                      40
alternative rock          28
permanent wave            20
mellow gold               19
soft rock                 19
alternative metal         16
modern rock               15
folk rock                 14
classic rock              11
grunge                    11
post-grunge               11
album rock                10
pop rock                   9
nu metal                   8
adult standards            7
hard rock                  7
piano rock                 6
folk                       6
singer-songwriter          5
classic country pop        5
yacht rock                 5
pop punk                   5
modern power pop           4
country rock               3
heartland rock             3
funk metal                 3
funk rock                  3
glam rock                  2
supergroup                 2
neo mellow                 2
traditional folk           1
blues rock                 1
art rock      